In [1]:
%load_ext autoreload
%autoreload 2

from iob_ia.utils import segment, io_utils, visualise, classify
import os

In [2]:
path_tif = 'G:/20250211_MarcDu/20x_Testfile_405(DAPI)_488(CD45)_546(CD3)_647(CD31)/DAPI_ch3-20x.tif'
path_nd2 = 'G:/20250211_MarcDu/20x_Testfile_405(DAPI)_488(CD45)_546(CD3)_647(CD31)/3-20x.nd2'

In [3]:
# Specify the channels (0-based indexing)
dapi_ch = 0
cd45_ch = 1
# channels of interest
oste_ch = 2
cd31_ch = 3

In [4]:
#img, voxel_size = io_utils.read_tif(path_tif) #uses different metadata names than vsi..
img_nd2 = io_utils.read_image(path_nd2)

In [5]:
print(img_nd2.get_data().shape)
print(img_nd2.physical_pixel_sizes)

(4, 68, 2048, 2048)
PhysicalPixelSizes(Z=2.0, Y=0.43177598559324926, X=0.43177598559324926)


## Segment using cellpose

In [6]:
if os.path.exists(io_utils.gen_out_path(path_tif, 'cp_seg')):
    mask_cp = io_utils.read_labels(io_utils.gen_out_path(path_tif, 'cp_seg'))
else:
    mask_cp = segment.segment_3d_cellpose(img_nd2.get_data()[dapi_ch]) #, anisotropy=voxel_size[0]/voxel_size[1])
print(mask_cp.shape)

(68, 2048, 2048)


In [ ]:
# save labels
save_path = io_utils.gen_out_path(path_tif, 'cp_seg')
io_utils.save_labels(mask_cp, save_path)

save_path = io_utils.gen_out_path(path_tif, 'gfpCh')
io_utils.save_labels(img_gfp, save_path)

## Filter lable image on features
shape features: 'area, 'euler_number' and 'extent'
intensity features: "intensity_max", "intensity_mean", "intensity_min"

In [7]:
# Measure features of all labels created with cellpose, using calibrated units
#property_table = segment.measure_props(img_label=mask_cp, img_intensity=img_nd2.get_data(), voxel_size=img_nd2.physical_pixel_sizes)
property_table = segment.measure_props(img_label=mask_cp, voxel_size=img_nd2.physical_pixel_sizes)

In [8]:
label2remove = segment.filter_shape(mask_cp, 'projected_circularity', min_val=0.55, props_table=property_table)
label2remove = segment.filter_shape(mask_cp, 'area', min_val=900, labels_to_remove=label2remove, props_table=property_table) # (10x10x9)um^3

In [9]:
#label2remove = segment.filter_intensity(mask_cp, img_gfp, 'intensity_mean', min_val=4000, labels_to_remove=label2remove)
#print(lbls)

In [10]:
nuclei = segment.remove_label_objects(mask_cp, labels=label2remove, relabel=False)

In [ ]:
save_path = io_utils.gen_out_path(path_tif, 'filtered')
io_utils.save_labels(nuclei, save_path)

## Visualise

In [11]:
# add multi channel nd2
visualise.add_multichannel_image(img=img_nd2.get_data(), name='image', channel_names=img_nd2.channel_names, scale=img_nd2.physical_pixel_sizes)
# create features for napari label layer (specifying the props_table will not recalculate the props, i.e. show no intensity measurements)
features = visualise.create_napari_features(
    img_label=nuclei, img_intensity=img_nd2.get_data(),
    voxel_size=img_nd2.physical_pixel_sizes, props_table=property_table)
# Adding both the filtered nuclei and the raw cp masks
visualise.add_labels(nuclei, name='nuclei', scale=img_nd2.physical_pixel_sizes, features=features)
#visualise.add_labels(mask_cp, name='raw_cp_mask', scale=img_nd2.physical_pixel_sizes, features=features)

## Label mask modulations

In [14]:
# Create cell an cytoplasm mask from nuclei
nuclei, cells, cyto = segment.create_cell_cyto_masks(nuclei, expansion=5, shrinking=2, voxel_size=img_nd2.physical_pixel_sizes)

Shrinking nuclei took: 51.505308628082275
Creating cells took: 46.0259051322937
Creating cytoplasm took: 0.2626194953918457


In [15]:
show_napari_features = True
if show_napari_features:
    from time import time
    start = time()
    nuc_props = segment.measure_props(img_label=nuclei, img_intensity=img_nd2.get_data(), voxel_size=img_nd2.physical_pixel_sizes)
    cyto_props = segment.measure_props(img_label=cyto, img_intensity=img_nd2.get_data(), voxel_size=img_nd2.physical_pixel_sizes)
    cell_props = segment.measure_props(img_label=cells, img_intensity=img_nd2.get_data(), voxel_size=img_nd2.physical_pixel_sizes)
    nuc_feat = visualise.create_napari_features(props_table=nuc_props)
    cyto_feat = visualise.create_napari_features(props_table=cyto_props)
    cell_feat = visualise.create_napari_features(props_table=cell_props)
    print('Measuring compartment features took:', time() - start)
else:
    nuc_feat = None
    cyto_feat = None
    cell_feat = None

Measuring compartment features took: 12.130110740661621


In [16]:
visualise.add_labels(nuclei, name='nuc_shrunk', scale=img_nd2.physical_pixel_sizes, features=nuc_feat)
visualise.add_labels(cyto, name='cyto', scale=img_nd2.physical_pixel_sizes, features=cyto_feat)
visualise.add_labels(cells, name='cells', scale=img_nd2.physical_pixel_sizes, features=cell_feat)

## Classification

In [24]:
table = classify.create_base_table(
    img=img_nd2.get_data(), nuclei=nuclei, cyto=cyto, cells=cells,
    channel_names=img_nd2.channel_names, scale=img_nd2.physical_pixel_sizes
)


In [25]:
table_mod = classify.classify(table, prop='Nucleus: projected_area', classification='big', min_val=74)


Classification
        915
big;    427
Name: count, dtype: int64


In [26]:
table_mod2 = classify.classify(table_mod, prop='Nucleus: projected_perimeter', classification=None, min_val=25)


Classification
Nucleus: projected_perimeter+;        611
big;Nucleus: projected_perimeter+;    427
                                      304
Name: count, dtype: int64


In [23]:
big_nuclei = classify.show_by_class(table=table_mod2, class_names='big;Nucleus: projected_perimeter+', img_nuclei=nuclei, voxel_size=img_nd2.physical_pixel_sizes, color='blue')